In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from libraries import AEM_preproc, ProbEM
from matplotlib.colors import LogNorm, Normalize

In [2]:
Survey = AEM_preproc.Survey
Sounding = ProbEM.Sounding

# Re-create the survey object using the newly imported Survey class
gex_file_path = "20230616_10098_306HP_LM_MergeGates_HM_splinegates_final_Z_V3.gex"
survey = Survey()
survey.proc_gex(gex_file_path)


# --- GENERATE SYNTHETIC DATA ---
# Define dummy line and time
mock_iline = 100
mock_time = 0.0

# Initialize the Data object in the survey
survey.Data = AEM_preproc.Data()
survey.Data.runc_offset = 0.03  # Default relative uncertainty offset

# Create a MultiIndex for the mock sounding
index = pd.MultiIndex.from_tuples([(mock_iline, mock_time)], names=["LINE_NO", "TIME"])

# Create synthetic Station Data (Position & Geometry)
survey.Data.station_data = pd.DataFrame(
    {
        "UTMX": [500000.0],
        "UTMY": [7000000.0],
        "ELEVATION": [100.0],
        "TX_ALTITUDE": [35.0],
        "RX_ALTITUDE": [35.0],
    },
    index=index,
)

In [3]:
# Initialize survey.Data with dummy LM/HM data and stds *before* the first mock_sounding instantiation
# These will be overwritten later with actual noisy synthetic data.
survey.Data.lm_data = pd.DataFrame(np.zeros((1, survey.n_lm_gates)), index=index)
survey.Data.hm_data = pd.DataFrame(np.zeros((1, survey.n_hm_gates)), index=index)
survey.Data.lm_std = pd.DataFrame(np.ones((1, survey.n_lm_gates)), index=index)
survey.Data.hm_std = pd.DataFrame(np.ones((1, survey.n_hm_gates)), index=index)


# Define the inverse thickness
inv_thickness = np.logspace(0, 1.5, 30)
inv_thickness = np.array(
    [
        2.0,
        2.2,
        2.4,
        2.6,
        2.8,
        3.1,
        3.4,
        3.7,
        4.0,
        4.4,
        4.8,
        5.2,
        5.6,
        6.2,
        6.7,
        7.3,
        8.0,
        8.7,
        9.5,
        10.4,
        11.3,
        12.3,
        13.4,
        14.6,
        16.0,
        17.4,
        19.0,
        20.7,
        22.6,
    ]
)


# Instantiate a Sounding object initially to get Depths, before defining true conductivity model
mock_sounding = Sounding(survey, mock_iline, mock_time, inv_thickness)

# 1. Initialize an array named `true_conductivity` with 0.05 S/m for all layers
true_conductivity = 0.05 * np.ones_like(mock_sounding.Depths)

# 2. Define the conductor's top depth as 20m and its bottom depth as 30m
conductor_top_depth = 20.0
conductor_bottom_depth = 30.0

# Identify the indices in mock_sounding.Depths that fall within the conductor's depth range
layer_tops = np.concatenate(([0.0], mock_sounding.Depths[:-1]))
layer_bottoms = mock_sounding.Depths

conductor_indices = np.where(
    (layer_tops < conductor_bottom_depth) & (layer_bottoms > conductor_top_depth)
)[0]

# 4. Set the `true_conductivity` values for these identified layers to 0.5 S/m
true_conductivity[conductor_indices] = 0.3

# 2. Define the conductor's top depth as 20m and its bottom depth as 30m
conductor_top_depth = 70.0
conductor_bottom_depth = 90.0

# Identify the indices in mock_sounding.Depths that fall within the conductor's depth range
layer_tops = np.concatenate(([0.0], mock_sounding.Depths[:-1]))
layer_bottoms = mock_sounding.Depths

conductor_indices = np.where(
    (layer_tops < conductor_bottom_depth) & (layer_bottoms > conductor_top_depth)
)[0]

true_conductivity[conductor_indices] = 0.3


# Set the true model in the simulation object
m_true = np.log(true_conductivity)
mock_sounding.simulation.m_true = m_true

synthetic_dobs = mock_sounding.simulation.make_synthetic_data(np.exp(m_true))


# Define the relative error based on survey.Data.runc_offset
relative_error_percentage = survey.Data.runc_offset


# The standard deviation must include the noise floor used in inversion
noise_floor = 1e-15
std_dev = np.sqrt(
    (synthetic_dobs.dobs * relative_error_percentage) ** 2 + noise_floor**2
)

# Add Gaussian noise to the synthetic data
# The mean of the noise distribution is 0, and std_dev provides the standard deviation for each point
noisy_dobs = synthetic_dobs.dobs + np.random.normal(
    0, std_dev, size=synthetic_dobs.dobs.shape
)

# Ensure data remains negative as it represents dB/dt responses
noisy_dobs = -np.abs(noisy_dobs)

# Split the noisy data and standard deviations back into LM and HM components
n_lm_gates = survey.n_lm_gates
n_hm_gates = survey.n_hm_gates

noisy_lm_data = noisy_dobs[:n_lm_gates]
noisy_hm_data = noisy_dobs[n_lm_gates:]

lm_std_data = std_dev[:n_lm_gates]
hm_std_data = std_dev[n_lm_gates:]

# Update the survey object's Data with the noisy data and calculated standard deviations
# Ensure the dataframes maintain their original MultiIndex
original_index = pd.MultiIndex.from_tuples(
    [(mock_iline, mock_time)], names=["LINE_NO", "TIME"]
)

survey.Data.lm_data = pd.DataFrame(noisy_lm_data.reshape(1, -1), index=original_index)
survey.Data.hm_data = pd.DataFrame(noisy_hm_data.reshape(1, -1), index=original_index)
survey.Data.lm_std = pd.DataFrame(lm_std_data.reshape(1, -1), index=original_index)
survey.Data.hm_std = pd.DataFrame(hm_std_data.reshape(1, -1), index=original_index)

print("Noisy synthetic data generated and updated in survey.Data.")


# Re-instantiate the Sounding object with the updated survey data
# This will automatically pick up the new noisy data and std_devs from survey.Data

mock_sounding = Sounding(survey, mock_iline, mock_time, inv_thickness)


Noisy synthetic data generated and updated in survey.Data.


In [ ]:
print("Running stochastic inversion with updated calculations (10 realizations)...")
mock_sounding.get_RML_reals(nreals=200)
mock_sounding.RML.run_local()

Running stochastic inversion with updated calculations (10 realizations)...
http://127.0.0.1:8787/status


In [ ]:
# --- 3. Visualize Results ---
# Verify layprob existence and shape
if hasattr(mock_sounding.RML, "layprob"):
    print(f"layprob calculated. Shape: {mock_sounding.RML.layprob.shape}")

    fig, ax1 = plt.subplots(figsize=(5, 10))

    # Plot Conductivity
    # Use correct depths from mock_sounding
    depths = mock_sounding.Depths

    # Check if dimensions match and adjust if necessary
    if len(mock_sounding.RML.p50) == len(depths):
        plot_depths_cond = depths
    else:
        # Fallback if shapes mismatch (should not happen with correct Depths)
        print(
            f"Warning: p50 shape {mock_sounding.RML.p50.shape} != depths shape {depths.shape}"
        )
        plot_depths_cond = depths[: len(mock_sounding.RML.p50)]

    ax1.step(mock_sounding.RML.p50, plot_depths_cond, "k-", label="P50 Conductivity")
    ax1.step(true_conductivity, mock_sounding.Depths, c="blue")
    ax1.fill_betweenx(
        plot_depths_cond,
        mock_sounding.RML.p5,
        mock_sounding.RML.p95,
        color="gray",
        alpha=0.3,
        label="P5-P95 Range",
        step="pre",
    )
    ax1.set_ylabel("Depth (m)")
    ax1.set_xlabel("Conductivity (S/m)")
    # ax1.set_xscale("log")
    ax1.invert_yaxis()
    ax1.grid(True, which="both", alpha=0.3)
    ax1.legend(loc="lower left")

    # Plot Layprob on twin axis
    ax2 = ax1.twiny()
    # layprob is derived from diff, so it typically has length N-1.
    # We plot it against the bottom depths of the first N-1 layers (or align as appropriate)
    layprob = mock_sounding.RML.layprob

    if len(layprob) == len(depths):
        plot_depths_prob = depths
    elif len(layprob) == len(depths) - 1:
        plot_depths_prob = depths[:-1]
    else:
        # Flexible matching
        plot_depths_prob = depths[: len(layprob)]

    ax2.step(
        mock_sounding.RML.pprob,
        plot_depths_prob,
        "r--",
        label="pprob",
    )
    ax2.set_xlabel("Probability", color="r")
    ax2.tick_params(axis="x", labelcolor="r")
    ax2.set_xlim(-1, 1)
    ax2.legend(loc="lower right")
    ax1.set_xscale("log")
    plt.title(f"Recovered Model & Layer Probability (Line {mock_sounding.iline})")
    plt.show()
else:
    print("Error: 'layprob' attribute not found in mock_sounding.RML.")


In [ ]:
plt.step(mock_sounding.RML.layer_index, mock_sounding.Depths)

In [ ]:
nreals = 1000

# mock_sounding = Sounding(survey, mock_iline, mock_time, inv_thickness)
# mock_sounding.get_RML_reals(nreals=nreals, Lrange=5)
dif_1 = [
    np.gradient(mock_sounding.RML.fields[x], mock_sounding.Depths)
    for x in range(nreals)
]


# Rising Probability (Conductivity Increasing with Depth)
ri_prob = np.sum([dif_1[x] > 0 for x in range(nreals)], axis=0) / nreals
ri_prob[ri_prob < 0.6] = 0.5

# Falling Probability (Conductivity Decreasing with Depth)
fa_prob = np.sum([dif_1[x] < 0 for x in range(nreals)], axis=0) / nreals
fa_prob[fa_prob < 0.6] = 0.5

# Combined Layer Probability (Directional Contrast)
layprob_p = ri_prob - fa_prob  # np.where((ri_prob > 0.6), ri_prob, fa_prob * -1)


# plt.step(np.array(mock_sounding.RML.fields).T, mock_sounding.Depths)
# plt.step(ri_prob, mock_sounding.Depths)
# plt.step(fa_prob, mock_sounding.Depths)
plt.step(ri_prob - fa_prob, mock_sounding.Depths)
plt.xlim(-0.5, 0.5)

nreals = 500

# mock_sounding = Sounding(survey, mock_iline, mock_time, inv_thickness)
# mock_sounding.get_RML_reals(nreals=nreals, Lrange=5)
dif_1 = [
    np.gradient(mock_sounding.RML.calreals[x], mock_sounding.Depths)
    for x in range(nreals)
]


# Rising Probability (Conductivity Increasing with Depth)
ri_prob = np.sum([dif_1[x] > 0 for x in range(nreals)], axis=0) / nreals
ri_prob[ri_prob < 0.6] = 0.5

# Falling Probability (Conductivity Decreasing with Depth)
fa_prob = np.sum([dif_1[x] < 0 for x in range(nreals)], axis=0) / nreals
fa_prob[fa_prob < 0.6] = 0.5

# Combined Layer Probability (Directional Contrast)
layprob_p = ri_prob - fa_prob  # np.where((ri_prob > 0.6), ri_prob, fa_prob * -1)


# plt.step(np.array(mock_sounding.RML.fields).T, mock_sounding.Depths)

# plt.step(fa_prob, mock_sounding.Depths)
threshold = 0.05
trans_up_prob = np.sum([dif_1[x] > threshold for x in range(nreals)], axis=0) / nreals

# Falling Transition (Must be negative AND steep)
trans_down_prob = (
    np.sum([dif_1[x] < -threshold for x in range(nreals)], axis=0) / nreals
)

tdiff = trans_up_prob - trans_down_prob
# tdiff[tdiff < 0.5] = 0
plt.step(tdiff, mock_sounding.Depths)
# plt.step(, mock_sounding.Depths)
plt.xlim(-1, 1)
plt.step(true_conductivity, mock_sounding.Depths, c="blue")

In [ ]:
# ... inside your RML class ...


nreals = len(mock_sounding.RML.calreals)
n_depths = len(mock_sounding.Depths)

# Initialize arrays to store counts
peak_counts = np.zeros(n_depths)
trough_counts = np.zeros(n_depths)

# Define Thresholds (Tune these!)
# Prominence: 0.1 = Peak must rise 0.1 decades (approx 25%) above background
# Width: 2 = Peak must cover at least 2 depth cells (removes spikes)
MIN_PROMINENCE = 0.1
MIN_WIDTH = 3

for real in mock_sounding.RML.calreals:
    # Convert to Log10 for consistent scale processing
    log_real = np.log10(real + 1e-8)

    # 1. FIND PEAKS (Conductive Layers)
    idx_peaks, _ = find_peaks(log_real, prominence=MIN_PROMINENCE, width=MIN_WIDTH)
    peak_counts[idx_peaks] += 1

    # 2. FIND TROUGHS (Resistive Layers)
    # We find troughs by finding peaks on the negative signal
    idx_troughs, _ = find_peaks(-log_real, prominence=MIN_PROMINENCE, width=MIN_WIDTH)
    trough_counts[idx_troughs] += 1

# Normalize by number of realizations
pprob = peak_counts / nreals
trough_prob = trough_counts / nreals

# 3. UPDATED "LAYER STATE" INDEX
# Now we include troughs too!
# +1 region: Rising -> Peak -> (High Conductivity)
# -1 region: Falling -> Trough -> (High Resistivity)

# Note: This is a complex combination.
# A simpler version is often just:
feature_index = pprob - trough_prob

In [ ]:
threshold = 0.05

# 2. Calculate Gradient Magnitude for all realizations
grad_mag = np.abs(dif_1)  # Absolute value of the gradient

# 3. Calculate "Transition Probability" (P_trans)
# "What % of realizations show a SHARP change here?"
tprob = np.sum([grad_mag[x] > threshold for x in range(nreals)], axis=0) / nreals
polarity_index = ri_prob - fa_prob

# 5. THE COMBINATION: Structural Intensity Index
# Range: -1 (Sharp Resistor) to +1 (Sharp Conductor)
# 0 means either "Flat" OR "Uncertain"
struct_index = polarity_index * tprob


peaks = []
for real in mock_sounding.RML.calreals:
    # Shifted arrays to compare neighbors
    # center > left AND center > right
    is_peak = np.r_[False, (real[1:-1] > real[:-2]) & (real[1:-1] > real[2:]), False]
    peaks.append(is_peak)

pprob = np.sum(peaks, axis=0) / nreals

# 3. COMBINED "LAYER STATE" INDEX
# (+1 = Top/Center of Conductor, -1 = Bottom of Conductor)
layer_index = ri_prob + pprob - fa_prob

# plt.step(struct_index, mock_sounding.Depths)

plt.step(layer_index, mock_sounding.Depths)
# plt.step(, mock_sounding.Depths)
plt.xlim(-1, 1)
plt.step(true_conductivity, mock_sounding.Depths, c="blue")
plt.step(feature_index, mock_sounding.Depths, c="purple")

In [ ]:
np.mean(mock_sounding.RML.prior_matrix, axis=0).T

In [ ]:
mock_sounding.inv_thickness

In [ ]:
plt.step(mock_sounding.RML.prior_matrix[33], mock_sounding.Depths)


In [ ]:
plt.step(mock_sounding.RML.p95, plot_depths_cond, "k-", label="P50 Conductivity")
plt.step(true_conductivity, mock_sounding.Depths, c="blue", label="blue")
# plt.xscale("log")

In [ ]:
plt.step(mock_sounding.RML.prior_matrix.T.mean(axis=1), mock_sounding.Depths)

In [ ]:
plt.step(mock_sounding.RML.pprob, mock_sounding.Depths * -1, c="r")
# plt.step(mock_sounding.RML.tprob, mock_sounding.Depths * -1)
plt.step(true_conductivity, mock_sounding.Depths * -1, c="blue")
plt.step(mock_sounding.RML.pprob, mock_sounding.Depths * -1, c="green")
plt.step(mock_sounding.RML.trough_prob * -1, mock_sounding.Depths * -1, c="red")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize


def plot_posterior_conductivity(
    realizations,
    layer_thicknesses,
    n_bins=100,
    cond_range=(0.001, 2.0),  # Default: 1 mS/m to 2 S/m
    figsize=(7, 10),
):
    """
    Plots the posterior probability density of CONDUCTIVITY (S/m).
    """
    n_reals, n_layers_data = realizations.shape
    n_layers_geo = len(layer_thicknesses)

    # --- 1. Fix Dimension Mismatch (Data vs Geology) ---
    if n_layers_data == n_layers_geo + 1:
        # Add dummy thickness for the infinite basement
        plot_thicknesses = np.r_[layer_thicknesses, 50.0]
    elif n_layers_data == n_layers_geo:
        plot_thicknesses = layer_thicknesses
    else:
        raise ValueError(
            f"Shape Mismatch: Data {n_layers_data} layers vs Geometry {n_layers_geo}."
        )

    # --- 2. Construct Grid Boundaries ---
    # Y-axis: Depth Edges
    depth_edges = np.r_[0, np.cumsum(plot_thicknesses)]
    if np.isinf(depth_edges[-1]):
        depth_edges[-1] = depth_edges[-2] + 50

    # X-axis: Conductivity Edges (Log Space)
    # We create bins from 1 mS/m to 2 S/m (or whatever you pass in)
    cond_bins_edges = np.logspace(
        np.log10(cond_range[0]), np.log10(cond_range[1]), n_bins
    )

    # --- 3. Compute Histogram ---
    # Shape: (n_layers, n_bins-1)
    pdf_matrix = np.zeros((n_layers_data, n_bins - 1))

    for i in range(n_layers_data):
        hist, _ = np.histogram(realizations[:, i], bins=cond_bins_edges, density=True)
        pdf_matrix[i, :] = hist

    # --- 4. Plotting ---
    fig, ax = plt.subplots(figsize=figsize)

    cmesh = ax.pcolormesh(
        cond_bins_edges,
        depth_edges,
        pdf_matrix,
        shading="flat",
        cmap="turbo",
        norm=Normalize(vmin=0, vmax=np.percentile(pdf_matrix, 99)),
    )

    # --- 5. Overlays (Statistics) ---
    p10 = np.percentile(realizations, 10, axis=0)
    p50 = np.percentile(realizations, 50, axis=0)
    p90 = np.percentile(realizations, 90, axis=0)

    # Plot Step Lines
    # We use 'post' to draw the value across the layer, then step down
    ax.step(p50, depth_edges[:-1], where="post", color="white", lw=2, label="Median")
    ax.step(
        p10,
        depth_edges[:-1],
        where="post",
        color="white",
        lw=1,
        ls="--",
        alpha=0.7,
        label="P10/P90",
    )
    ax.step(
        p90, depth_edges[:-1], where="post", color="white", lw=1, ls="--", alpha=0.7
    )

    # --- Formatting ---
    ax.set_xscale("log")
    ax.set_ylim(depth_edges[-1], 0)  # Invert Y (Depth down)
    ax.set_xlim(cond_range)

    ax.set_xlabel("Conductivity (S/m)", fontsize=12)
    ax.set_ylabel("Depth (m)", fontsize=12)
    ax.set_title(f"Posterior Conductivity ({n_reals} Realizations)", fontsize=14)

    cbar = plt.colorbar(cmesh, ax=ax, pad=0.05)
    cbar.set_label("Probability Density")

    ax.legend(loc="lower left", facecolor="black", framealpha=0.5, labelcolor="white")
    ax.grid(True, which="both", alpha=0.2)

    plt.tight_layout()
    plt.show()

In [ ]:
np.quantile(mock_sounding.RML.prior_matrix.T, 0.05, axis=1)


In [ ]:
plt.step(mock_sounding.RML.prior_matrix.T, mock_sounding.Depths, c="grey", alpha=0.5)
plt.xscale("log")

In [ ]:
fig, ax = plt.subplots()
plt.step(mock_sounding.RML.p50, mock_sounding.Depths)
plt.step(mock_sounding.RML.p5, mock_sounding.Depths)
plt.step(mock_sounding.RML.p95, mock_sounding.Depths)
plt.step(mock_sounding.RML.p95, mock_sounding.Depths)
ax.invert_yaxis()


In [ ]:
np.array(mock_sounding.RML.calreals).T


In [ ]:
plt.hist(np.array(mock_sounding.RML.prior_matrix).T[0, :])


In [ ]:
data_to_plot = np.log10(np.array(mock_sounding.RML.prior_matrix))

# 2. Check & Convert Units
# If the median value is small (e.g., < 5), it's definitely Log10.
# The plotter needs Linear units (Ohm-m) because it calculates log-bins internally.

# 3. Plot
plot_posterior_conductivity(
    realizations=data_to_plot,  # (2000, 30)
    layer_thicknesses=inv_thickness,  # Your 30-layer geometry array
    n_bins=100,
    cond_range=(0.1, 3),  # Adjust min/max Ohm-m to fit your geology
    figsize=(7, 10),
)


In [ ]:
def get_prior_reals_CONV(self, Sounding, nreals):
    """
    Generates 'Background + Anomalies' priors using Matrix Convolution.
    Produces peaks/troughs oscillating around a fixed background (ival).
    """
    # 1. Grab Depths
    self.Depths = Sounding.Depths
    self.nreals = nreals

    # 2. Setup Geometry (Layer Midpoints)
    z_bot = Sounding.inv_thickness.cumsum()
    z_top = np.r_[0, z_bot[:-1]]
    z_mid = (z_top + z_bot) / 2.0

    # Fix dimensions for SimPEG (30 cells)
    n_cells = Sounding.mesh.nC
    if len(z_mid) == n_cells - 1:
        z_mid = np.r_[z_mid, z_bot[-1] + 50.0]
    if len(z_mid) != n_cells:
        z_mid = np.linspace(0, z_bot[-1] + 100, n_cells)

    # --- 3. DEFINE FEATURE WIDTH (Correlation Length) ---
    # To avoid trends, these numbers must be significantly smaller than total depth.
    # 10m at surface -> 40m at depth creates distinct "layers" rather than a drift.
    L_surface = 30.0
    L_bottom = 80.0

    length_scales = np.interp(z_mid, [0, 250], [L_surface, L_bottom])

    # --- 4. BUILD CORRELATION MATRIX (Exponential Kernel) ---
    # Exponential (blockier) vs Gaussian (smoother).
    # Exponential is often better for "peaks and troughs".
    nC = len(z_mid)
    C = np.eye(nC)

    for i in range(nC):
        for j in range(nC):
            if i == j:
                continue
            dist = abs(z_mid[i] - z_mid[j])
            L_local = (length_scales[i] + length_scales[j]) / 2.0

            # Exponential Kernel: exp( - distance / length )
            # Produces "rougher" layers (Markov process)
            C[i, j] = np.exp(-1.0 * (dist / L_local))

    # --- 5. CHOLESKY DECOMPOSITION ---
    # Add jitter for stability
    C += np.eye(nC) * 1e-6
    try:
        L_mat = np.linalg.cholesky(C)
    except np.linalg.LinAlgError:
        U, S, Vt = np.linalg.svd(C)
        L_mat = U @ np.diag(np.sqrt(S))

    # --- 6. GENERATE ---
    # Calculate Log-Space Statistics
    log_lower = np.log10(self.lower)
    log_upper = np.log10(self.upper)
    var_log = ((log_upper - log_lower) / 6.0) ** 2
    std_log = np.sqrt(var_log)

    # Center the mean exactly on your background (ival)
    # We assume the output of the convolution is mean=0, so we just add log(ival)
    log_background = np.log10(self.ival)

    self.fields = []
    MIN_LOG_COND = -4.0
    MAX_LOG_COND = 0.5

    for _ in range(self.nreals):
        # Generate Fluctuations
        white_noise = np.random.randn(nC)
        fluctuations = L_mat @ white_noise

        # Combine: Background + (Fluctuation * Magnitude)
        log_model = log_background + (fluctuations * std_log)

        # Clip and Convert to Linear S/m
        self.fields.append(10 ** (np.clip(log_model, MIN_LOG_COND, MAX_LOG_COND)))

    self.prior_matrix = np.array(self.fields)


In [ ]:
get_prior_reals_CONV(mock_sounding.RML, mock_sounding, 2000)

In [ ]:
plt.step(mock_sounding.RML.prior_matrix[28], mock_sounding.Depths)